In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1 pyspark-shell'

In [2]:

kafka_servers = "localhost:9092"

import findspark
findspark.init('/home/lexzhe/spark/spark-3.1.1-bin-hadoop3.2')

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
spark = (SparkSession
        .builder
        .appName("consumer_structured_streaming_booksratings_1")  
        .getOrCreate())

23/04/22 18:49:08 WARN Utils: Your hostname, DESKTOP-VAUHEFB resolves to a loopback address: 127.0.1.1; using 172.25.90.44 instead (on interface eth0)
23/04/22 18:49:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/lexzhe/.ivy2/cache
The jars for the packages stored in: /home/lexzhe/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-08d2e6b3-e48e-4e27-becd-e07ca87a6ad6;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/lexzhe/spark/spark-3.1.1-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.1 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 232ms :: artifacts dl 5ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central in [default]
	org.apache.kafka#kafka-clients;2.6.0 from central in [default]
	org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 from central in [default]
	org.apache.spark#spark-streaming-kafka-0-10_2.12;3.1.1 from central in [def

In [4]:
topicInput = "booksRatingsInput"
topicOutput = "booksRatingsOutput"
checkpoint_path = "./checkpoint/ratings"

In [7]:
ratingsDF = spark \
            .readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", kafka_servers) \
            .option("subscribe",topicInput) \
            .option("startingOffsets", "earliest") \
            .load() \
            .selectExpr("CAST(value AS STRING)") 

schema = StructType([StructField('name', StringType(), True), \
                   StructField('rating', IntegerType(), True)])


query = ratingsDF.select(from_json(col("value"), schema).alias("t")) \
            .select('t.name', 't.rating') \
            .groupBy('name').mean('rating') \
            .withColumnRenamed('avg(rating)', 'average_rating') \
            .selectExpr("CAST(name AS STRING) AS key", "to_json(struct(*)) AS value") \
            .writeStream.format("kafka") \
            .option("kafka.bootstrap.servers", kafka_servers) \
            .option("truncate", "false") \
            .option("topic", topicOutput) \
            .outputMode("update") \
            .option("checkpointLocation", checkpoint_path) \
            .start() \
            .awaitTermination()
    

23/04/22 18:50:18 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-1c15be64-ed6a-4915-a671-b25581bb13f3-1465316401-driver-0-1, groupId=spark-kafka-source-1c15be64-ed6a-4915-a671-b25581bb13f3-1465316401-driver-0] Error while fetching metadata with correlation id 2 : {booksRatingsInput=LEADER_NOT_AVAILABLE}
23/04/22 18:51:06 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 3 : {booksRatingsOutput=LEADER_NOT_AVAILABLE}


KeyboardInterrupt: 